<a href="https://colab.research.google.com/github/Suryansh-gp/DEVA_AI/blob/main/DEVA_AI_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 23.5 MB/s eta 0:00:00
   

***Libraries***

In [ ]:
import pandas as pd
import gradio as gr
from crewai import Agent
import os

# Set the environment variable for OpenAI API Key
os.environ["OPENAI_API_KEY"] = ""

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


***Problem Identifier Agent***

In [ ]:
# Problem Identifier Agent
class ProblemIdentifierAgent:
    def __init__(self, role, goal, backstory):
        self.role = role
        self.goal = goal
        self.backstory = backstory

    def run(self, dataset_details):
        dataset_name = dataset_details['name']
        dataset_columns = dataset_details['columns']

        problem_statements = self.analyze_data(dataset_columns)

        for problem in problem_statements:
            problem['suggested_algorithms'] = self.suggest_algorithms(problem['statement'])

        return {
            "dataset_name": dataset_name,
            "problem_statements": problem_statements
        }

    def analyze_data(self, columns):
        problem_statements = []

        # Customer Churn Prediction
        if all(feature in columns for feature in ["tenure", "Contract", "MonthlyCharges", "TotalCharges", "InternetService", "OnlineSecurity", "TechSupport", "PaymentMethod", "Churn"]):
            problem_statements.append({
                "statement": "Predict customer churn based on tenure, contract type, and service usage."
            })

        # Customer Segmentation for Personalized Marketing
        if all(feature in columns for feature in ["gender", "SeniorCitizen", "Partner", "Dependents", "tenure", "PhoneService", "MultipleLines", "InternetService", "StreamingTV", "StreamingMovies", "Contract", "PaymentMethod", "MonthlyCharges"]):
            problem_statements.append({
                "statement": "Segment customers for personalized marketing based on demographics and service usage."
            })

        # Service Quality Improvement and Complaint Reduction
        if all(feature in columns for feature in ["TechSupport", "DeviceProtection", "OnlineSecurity", "InternetService", "MultipleLines", "StreamingTV", "StreamingMovies", "Churn"]):
            problem_statements.append({
                "statement": "Analyze service features to improve quality and reduce customer complaints."
            })

        # Customer Lifetime Value (CLV) Prediction
        if all(feature in columns for feature in ["tenure", "Contract", "MonthlyCharges", "TotalCharges", "PaymentMethod", "InternetService", "StreamingTV", "StreamingMovies"]):
            problem_statements.append({
                "statement": "Predict customer lifetime value based on service usage and contract details."
            })

        return problem_statements[:3]

    def suggest_algorithms(self, problem_statement):
        # Suggest classification algorithms based on the problem statement
        if "churn" in problem_statement.lower():
            return ["Logistic Regression", "Random Forest", "Support Vector Machine"]
        elif "segment" in problem_statement.lower():
            return ["K-Means Clustering", "Hierarchical Clustering"]
        elif "service quality" in problem_statement.lower():
            return ["Decision Trees", "Random Forest", "Gradient Boosting"]
        elif "lifetime value" in problem_statement.lower():
            return ["Linear Regression", "Random Forest", "XGBoost"]
        else:
            return ["Logistic Regression", "Naive Bayes", "K-Nearest Neighbors"]

# Function to process input and return output
def process_input(dataset_columns_input):
    dataset_columns = [column.strip() for column in dataset_columns_input.split(",")]

    dataset_details = {
        "name": "Default Dataset Name",
        "description": "Default Dataset Description",
        "columns": dataset_columns
    }

    # Instantiate the agent with required parameters
    role = "Problem Statement Identifier"
    goal = "Identify problem statements related to customer churn and suggest algorithms."
    backstory = "This agent analyzes dataset columns to generate relevant problem statements."

    agent = ProblemIdentifierAgent(role, goal, backstory)
    results = agent.run(dataset_details)

    output = ""
    for problem in results['problem_statements']:
        output += f"""
        <div style="border: 1px solid #007BFF; border-radius: 8px; padding: 16px; margin: 8px; width: 300px; display: inline-block; background-color: black; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
            <h4 style="color: #007BFF;">Problem Statement:</h4>
            <p style="color:white;">{problem['statement']}</p>
            <h5 style="color: #28a745;">Suggested Algorithms:</h5>
            <p>{', '.join(problem['suggested_algorithms'])}</p>
        </div>
        """

    return output

iface = gr.Interface(
    fn=process_input,
    inputs=[
        gr.Textbox(label="Dataset Features (comma-separated)")
    ],
    outputs=gr.HTML(),
    title="DEVA AI",
    description="## Problem Identifier Agent\nEnter the column names to identify relevant problem statements and suggested algorithms.",
    theme="default"
)

if __name__ == "__main__":
    iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e7c90c2ceb0868230f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
#gr.close_all()

***Data Ingestion Agent***

In [ ]:
# Data Ingestion Agent
class DataIngestionAgent(Agent):
    def __init__(self, role, goal, backstory):
        super().__init__(role=role, goal=goal, backstory=backstory)

    def load_data(self, file_url):
        try:
            # Check the file extension
            if file_url.endswith('.csv'):
                telecom_data = pd.read_csv(file_url)
            elif file_url.endswith('.xlsx'):
                telecom_data = pd.read_excel(file_url)
            else:
                raise ValueError("Unsupported file extension. Please provide a .csv or .xlsx file.")

            print("\nDataset loaded successfully.")
            return telecom_data
        except Exception as e:
            print(f"Error loading data: {e}")
            return None

    def explore_data(self, telecom_data):
        print("\nPreview of the dataset:\n")
        display(telecom_data.head(10))

        print("\nBasic statistics:\n")
        display(telecom_data.describe())

        print(f"\nNumber of rows: {telecom_data.shape[0]}")
        print(f"\nNumber of columns: {telecom_data.shape[1]}")

        # Display data types
        print("\nData Info:")
        display(telecom_data.info())

    def run(self, file_url):
        telecom_data = self.load_data(file_url)

        if telecom_data is not None:
            self.explore_data(telecom_data)
            return telecom_data
        else:
            return None


if __name__ == "__main__":
    dataset_url = "https://raw.githubusercontent.com/Suryansh-gp/DEVA_AI/main/Telecom_Churn.csv"

    role = "Data Ingestion Agent"
    goal = "Ingest the telecom customer churn dataset."
    backstory = "This agent loads and explores the dataset for analysis."

    agent = DataIngestionAgent(role, goal, backstory)

    telecom_data = agent.run(dataset_url)


Dataset loaded successfully.

Preview of the dataset:



,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No



Basic statistics:



,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000



Number of rows: 7043

Number of columns: 21

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null 

None